<a href="https://colab.research.google.com/github/ceying/DataSparsityProject_CreditStability/blob/main/HomeCreditStability_RunBestAnalysis_March2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score, balanced_accuracy_score

np.random.seed(0)

In [ ]:
# display all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Prep and concat results


## Load & concat pred_proba from each model
new_full_results_df.csv

In [ ]:
DT_results = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_DT_final_results_df.csv')
RF_results = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_RF_final_results_df.csv')
LGBM_results = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_LGBM_final_results_df.csv')
XGB_results = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_XGB_final_results_df.csv')


In [ ]:
DT_results.shape

(959900, 6)

In [ ]:
DT_results.head()

,y_test,y_pred_proba,test_index,iteration,completeness,completeness_level
0,1,0.731983,24203,0,62.247839,mid
1,1,0.775148,77881,0,80.691643,high
2,0,0.113986,50081,0,90.201729,high
3,0,0.113986,52555,0,88.472622,high
4,1,0.690063,91382,0,80.115274,high


In [ ]:
# prompt: for files new_DT_final_results_df.csv, new_RF_final_results_df.csv,  new_LGBM_final_results_df.csv, new_XGB_final_results_df.csv in file_path, save 'y_test', 'y_pred_proba' and rename column to include file name before "_final_results_df" to  a new df results_df, join on test_index and iteration

file_paths = {
    'DT': '/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_DT_final_results_df.csv',
    'RF': '/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_RF_final_results_df.csv',
    'LGBM': '/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_LGBM_final_results_df.csv',
    'XGB': '/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_XGB_final_results_df.csv'
}

results_df = pd.DataFrame()

for model_name, file_path in file_paths.items():
    temp_df = pd.read_csv(file_path)
    temp_df = temp_df[['y_test', 'y_pred_proba', 'test_index', 'iteration']]
    temp_df = temp_df.rename(columns={'y_test': f'{model_name}_y_test', 'y_pred_proba': f'{model_name}_y_pred_proba'})

    if results_df.empty:
        results_df = temp_df
    else:
        results_df = pd.merge(results_df, temp_df, on=['test_index', 'iteration'], how='left')

results_df


,DT_y_test,DT_y_pred_proba,test_index,iteration,RF_y_test,RF_y_pred_proba,LGBM_y_test,LGBM_y_pred_proba,XGB_y_test,XGB_y_pred_proba
0,1,0.731983,24203,0,1,0.749422,1,0.863152,1,0.960844
1,1,0.775148,77881,0,1,0.645641,1,0.804602,1,0.689335
2,0,0.113986,50081,0,0,0.172363,0,0.046961,0,0.027819
3,0,0.113986,52555,0,0,0.172207,0,0.116721,0,0.163814
4,1,0.690063,91382,0,1,0.585506,1,0.688801,1,0.841738
...,...,...,...,...,...,...,...,...,...,...
959895,1,0.900789,78211,49,1,0.905110,1,0.933936,1,0.967337
959896,0,0.148430,58097,49,0,0.062379,0,0.039504,0,0.032402
959897,0,0.640376,79,49,0,0.455657,0,0.430316,0,0.376093
959898,0,0.082632,54335,49,0,0.049435,0,0.035136,0,0.010333


In [ ]:
inconsistent_rows = results_df[
    results_df['DT_y_test'] != results_df['RF_y_test']
    | (results_df['DT_y_test'] != results_df['LGBM_y_test'])
    | (results_df['DT_y_test'] != results_df['XGB_y_test'])
    | (results_df['RF_y_test'] != results_df['LGBM_y_test'])
    | (results_df['RF_y_test'] != results_df['XGB_y_test'])
    | (results_df['LGBM_y_test'] != results_df['XGB_y_test'])
]

print("Number of inconsistent rows:", len(inconsistent_rows))
inconsistent_rows.head()


Number of inconsistent rows: 0


,DT_y_test,DT_y_pred_proba,test_index,iteration,RF_y_test,RF_y_pred_proba,LGBM_y_test,LGBM_y_pred_proba,XGB_y_test,XGB_y_pred_proba


In [ ]:
DT_results.shape
DT_results.head()

(959900, 6)

,y_test,y_pred_proba,test_index,iteration,completeness,completeness_level
0,1,0.731983,24203,0,62.247839,mid
1,1,0.775148,77881,0,80.691643,high
2,0,0.113986,50081,0,90.201729,high
3,0,0.113986,52555,0,88.472622,high
4,1,0.690063,91382,0,80.115274,high


In [ ]:
results_df.shape
results_df.head()

(959900, 10)

,DT_y_test,DT_y_pred_proba,test_index,iteration,RF_y_test,RF_y_pred_proba,LGBM_y_test,LGBM_y_pred_proba,XGB_y_test,XGB_y_pred_proba
0,1,0.731983,24203,0,1,0.749422,1,0.863152,1,0.960844
1,1,0.775148,77881,0,1,0.645641,1,0.804602,1,0.689335
2,0,0.113986,50081,0,0,0.172363,0,0.046961,0,0.027819
3,0,0.113986,52555,0,0,0.172207,0,0.116721,0,0.163814
4,1,0.690063,91382,0,1,0.585506,1,0.688801,1,0.841738


In [ ]:
# Rename 'DT_y_test' to 'y_test'
results_df = results_df.rename(columns={'DT_y_test': 'y_test'})

# Drop columns
results_df = results_df.drop(['RF_y_test', 'LGBM_y_test', 'XGB_y_test'], axis=1)

results_df.head()

,y_test,DT_y_pred_proba,test_index,iteration,RF_y_pred_proba,LGBM_y_pred_proba,XGB_y_pred_proba
0,1,0.731983,24203,0,0.749422,0.863152,0.960844
1,1,0.775148,77881,0,0.645641,0.804602,0.689335
2,0,0.113986,50081,0,0.172363,0.046961,0.027819
3,0,0.113986,52555,0,0.172207,0.116721,0.163814
4,1,0.690063,91382,0,0.585506,0.688801,0.841738


In [ ]:
# prompt: add 'completeness_level' from DT_results to results_df, merge on text_index and iteration

# Merge 'completeness_level' from DT_results to results_df
results_df = pd.merge(results_df, DT_results[['test_index', 'iteration', 'completeness_level']], on=['test_index', 'iteration'], how='left')

results_df.shape
results_df.head()


(959900, 8)

,y_test,DT_y_pred_proba,test_index,iteration,RF_y_pred_proba,LGBM_y_pred_proba,XGB_y_pred_proba,completeness_level
0,1,0.731983,24203,0,0.749422,0.863152,0.960844,mid
1,1,0.775148,77881,0,0.645641,0.804602,0.689335,high
2,0,0.113986,50081,0,0.172363,0.046961,0.027819,high
3,0,0.113986,52555,0,0.172207,0.116721,0.163814,high
4,1,0.690063,91382,0,0.585506,0.688801,0.841738,high


In [ ]:
balanced_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/full_balanced_df.csv')

<ipython-input-15-3dccdb10bda2>:1: DtypeWarning: Columns (261,272,289,294,326,327) have mixed types. Specify dtype option on import or set low_memory=False.
  balanced_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/full_balanced_df.csv')


In [ ]:
# prompt: for each iteration in results_df['iteration'], set_index(test_index) and merge with balanced_df, left_index=True, right_index=True, how='left'
full_results_df = pd.DataFrame()
for iteration in results_df['iteration'].unique():
    temp_df = results_df[results_df['iteration'] == iteration].set_index('test_index')
    temp_df = pd.merge(temp_df, balanced_df['case_id'], left_index=True, right_index=True, how='left')

    full_results_df = pd.concat([full_results_df, temp_df], axis=0)

In [ ]:
full_results_df

,y_test,DT_y_pred_proba,iteration,RF_y_pred_proba,LGBM_y_pred_proba,XGB_y_pred_proba,completeness_level,case_id
test_index,,,,,,,,
24203,1,0.731983,0,0.749422,0.863152,0.960844,mid,642035
77881,1,0.775148,0,0.645641,0.804602,0.689335,high,1539041
50081,0,0.113986,0,0.172363,0.046961,0.027819,high,1817081
52555,0,0.113986,0,0.172207,0.116721,0.163814,high,1435206
91382,1,0.690063,0,0.585506,0.688801,0.841738,high,2560708
...,...,...,...,...,...,...,...,...
78211,1,0.900789,49,0.905110,0.933936,0.967337,high,1546802
58097,0,0.148430,49,0.062379,0.039504,0.032402,high,2681238
79,0,0.640376,49,0.455657,0.430316,0.376093,low,837560


In [ ]:
len(full_results_df['case_id'].unique())

95988

In [ ]:
full_results_df.to_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_full_results_df.csv', index=False)

## Load and concat combined_metrics from each model
new_all_metrics.csv


In [ ]:
DT_metric = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_DT_combined_metrics_df.csv')
RF_metric = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_RF_combined_metrics_df.csv')
LGBM_metric = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_LGBM_combined_metrics_df.csv')
XGB_metric = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_XGB_combined_metrics_df.csv')


In [ ]:
DT_metric

,iteration,Metric,Value_low,Value_mid,Value_high
0,0,F1 Score,0.631229,0.674294,0.695968
1,0,Precision,0.580448,0.651371,0.674571
2,0,Recall,0.691748,0.698889,0.718766
3,0,ROC AUC,0.636415,0.731677,0.759342
4,0,Accuracy,0.604982,0.665974,0.689898
...,...,...,...,...,...
295,49,Precision,0.596311,0.664973,0.668356
296,49,Recall,0.659864,0.748871,0.749268
297,49,ROC AUC,0.602116,0.737153,0.753936
298,49,Accuracy,0.606129,0.687434,0.689433


In [ ]:
DT_metric.shape

(300, 5)

In [ ]:
# prompt: create a new dataframe to concat all metric tables DT_metric, RF_metric, LGBM_metric and XGB_metric and add a new column for model name from "_metric"

import pandas as pd

# Load the metric dataframes
DT_metric = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_DT_combined_metrics_df.csv')
RF_metric = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_RF_combined_metrics_df.csv')
LGBM_metric = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_LGBM_combined_metrics_df.csv')
XGB_metric = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_XGB_combined_metrics_df.csv')

# Add a column for the model name
DT_metric['Model'] = 'DT'
RF_metric['Model'] = 'RF'
LGBM_metric['Model'] = 'LGBM'
XGB_metric['Model'] = 'XGB'

# Concatenate the dataframes
all_metrics = pd.concat([DT_metric, RF_metric, LGBM_metric, XGB_metric], ignore_index=True)

# Display the combined dataframe
all_metrics


,iteration,Metric,Value_low,Value_mid,Value_high,Model
0,0,F1 Score,0.631229,0.674294,0.695968,DT
1,0,Precision,0.580448,0.651371,0.674571,DT
2,0,Recall,0.691748,0.698889,0.718766,DT
3,0,ROC AUC,0.636415,0.731677,0.759342,DT
4,0,Accuracy,0.604982,0.665974,0.689898,DT
...,...,...,...,...,...,...
1195,49,Precision,0.699332,0.742543,0.751403,XGB
1196,49,Recall,0.712018,0.772057,0.783882,XGB
1197,49,ROC AUC,0.756414,0.829135,0.839302,XGB
1198,49,Accuracy,0.702611,0.753482,0.762801,XGB


In [ ]:
all_metrics.to_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_all_metrics.csv', index=False)

# Analysis

In [ ]:
all_metrics = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CreditFairness/HomeCreditStability/Results/new50_all_metrics.csv')

In [ ]:
data = all_metrics

In [ ]:
import pandas as pd
from scipy import stats


# Reshape the data
data_long = pd.melt(data, id_vars=['Model', 'Metric'], value_vars=['Value_low', 'Value_mid', 'Value_high'], var_name='Group', value_name='Value')



In [ ]:
data_long
data_long.shape

,Model,Metric,Group,Value
0,DT,F1 Score,Value_low,0.631229
1,DT,Precision,Value_low,0.580448
2,DT,Recall,Value_low,0.691748
3,DT,ROC AUC,Value_low,0.636415
4,DT,Accuracy,Value_low,0.604982
...,...,...,...,...
3595,XGB,Precision,Value_high,0.751403
3596,XGB,Recall,Value_high,0.783882
3597,XGB,ROC AUC,Value_high,0.839302
3598,XGB,Accuracy,Value_high,0.762801


(3600, 4)

## Normality Test

In [ ]:
# Test for normality using Shapiro-Wilk and Kolmogorov-Smirnov tests
normality_test_sw = data_long.groupby(['Model', 'Metric', 'Group'])['Value'].apply(lambda x: stats.shapiro(x))
normality_test_ks = data_long.groupby(['Model', 'Metric', 'Group'])['Value'].apply(lambda x: stats.kstest(x, 'norm'))

# Extract test statistics and p-values
normality_results_sw = normality_test_sw.apply(lambda x: pd.Series({'statistic': x.statistic, 'p-value': x.pvalue}))
normality_results_ks = normality_test_ks.apply(lambda x: pd.Series({'statistic': x.statistic, 'p-value': x.pvalue}))

print("Shapiro-Wilk Normality Test Results:")
print(normality_results_sw)
print("\nKolmogorov-Smirnov Normality Test Results:")
print(normality_results_ks)

Shapiro-Wilk Normality Test Results:
                                    statistic   p-value
Model Metric            Group                          
DT    Accuracy          Value_high   0.969400  0.218786
                        Value_low    0.959515  0.084828
                        Value_mid    0.982683  0.669219
      Balanced Accuracy Value_high   0.968125  0.193967
                        Value_low    0.964168  0.132872
...                                       ...       ...
XGB   ROC AUC           Value_low    0.990616  0.959455
                        Value_mid    0.970011  0.231690
      Recall            Value_high   0.980278  0.563745
                        Value_low    0.986035  0.815032
                        Value_mid    0.960318  0.091659

[72 rows x 2 columns]

Kolmogorov-Smirnov Normality Test Results:
                                    statistic       p-value
Model Metric            Group                              
DT    Accuracy          Value_high   0.749356  1

In [ ]:
normality_results_sw

statistic   p-value
Model Metric            Group                          
DT    Accuracy          Value_high   0.969400  0.218786
                        Value_low    0.959515  0.084828
                        Value_mid    0.982683  0.669219
      Balanced Accuracy Value_high   0.968125  0.193967
                        Value_low    0.964168  0.132872
...                                       ...       ...
XGB   ROC AUC           Value_low    0.990616  0.959455
                        Value_mid    0.970011  0.231690
      Recall            Value_high   0.980278  0.563745
                        Value_low    0.986035  0.815032
                        Value_mid    0.960318  0.091659

[72 rows x 2 columns]

In [ ]:
normality_results_ks

statistic       p-value
Model Metric            Group                              
DT    Accuracy          Value_high   0.749356  1.523549e-29
                        Value_low    0.717492  1.375930e-26
                        Value_mid    0.746344  2.986183e-29
      Balanced Accuracy Value_high   0.749364  1.520619e-29
                        Value_low    0.718691  1.077797e-26
...                                       ...           ...
XGB   ROC AUC           Value_low    0.765336  3.823295e-31
                        Value_mid    0.794265  2.789073e-34
      Recall            Value_high   0.780350  9.909357e-33
                        Value_low    0.745480  3.617881e-29
                        Value_mid    0.776462  2.600797e-32

[72 rows x 2 columns]

In [ ]:
data_long

,Model,Metric,Group,Value
0,DT,F1 Score,Value_low,0.631229
1,DT,Precision,Value_low,0.580448
2,DT,Recall,Value_low,0.691748
3,DT,ROC AUC,Value_low,0.636415
4,DT,Accuracy,Value_low,0.604982
...,...,...,...,...
3595,XGB,Precision,Value_high,0.751403
3596,XGB,Recall,Value_high,0.783882
3597,XGB,ROC AUC,Value_high,0.839302
3598,XGB,Accuracy,Value_high,0.762801


In [ ]:
data

,iteration,Metric,Value_low,Value_mid,Value_high,Model
0,0,F1 Score,0.631229,0.674294,0.695968,DT
1,0,Precision,0.580448,0.651371,0.674571,DT
2,0,Recall,0.691748,0.698889,0.718766,DT
3,0,ROC AUC,0.636415,0.731677,0.759342,DT
4,0,Accuracy,0.604982,0.665974,0.689898,DT
...,...,...,...,...,...,...
1195,49,Precision,0.699332,0.742543,0.751403,XGB
1196,49,Recall,0.712018,0.772057,0.783882,XGB
1197,49,ROC AUC,0.756414,0.829135,0.839302,XGB
1198,49,Accuracy,0.702611,0.753482,0.762801,XGB


In [ ]:
# prompt: rename column Value_low to low, value_mid to mid and Value_high to high

# Rename columns
df = data.rename(columns={
    'Value_low': 'low',
    'Value_mid': 'mid',
    'Value_high': 'high'
})


In [ ]:
models = df["Model"].unique()
metrics = df["Metric"].unique()

## Pairwise t-test

In [ ]:
import pandas as pd
from scipy import stats

results = []

for metric in metrics:

    for model in models:
        # Filter the data
        df_filtered = df[(df["Metric"] == metric) & (df["Model"] == model)]

        # Perform the t-test
        for comparison in [("low", "mid"), ("low", "high"), ("mid", "high")]:
            values1 = df_filtered[(df_filtered["Metric"] == metric) & (df_filtered["Model"].isin(models))][comparison[0]].values
            values2 = df_filtered[(df_filtered["Metric"] == metric) & (df_filtered["Model"].isin(models))][comparison[1]].values

            # Perform the paired t-test
            t_stat, p_val = stats.ttest_rel(values1, values2)

            # Append the results to the list
            results.append({
                'Model': model,
                "Metric": metric,
                "Classes": f"{comparison[0]} vs {comparison[1]}",
                "t-stat": t_stat,
                "p-val": p_val
            })

# Convert the list to a DataFrame
df_results = pd.DataFrame(results)

In [ ]:
df_results

,Model,Metric,Classes,t-stat,p-val
0,DT,F1 Score,low vs mid,-9.263292,2.397203e-12
1,DT,F1 Score,low vs high,-12.114933,2.378574e-16
2,DT,F1 Score,mid vs high,-5.777868,5.137226e-07
3,RF,F1 Score,low vs mid,-25.306718,8.645334e-30
4,RF,F1 Score,low vs high,-28.671372,2.789143e-32
...,...,...,...,...,...
67,LGBM,Balanced Accuracy,low vs high,-46.090238,5.125130e-42
68,LGBM,Balanced Accuracy,mid vs high,-17.606774,7.855819e-23
69,XGB,Balanced Accuracy,low vs mid,-29.078383,1.451541e-32
70,XGB,Balanced Accuracy,low vs high,-37.173349,1.444046e-37


## One Way ANOVA

In [ ]:
results = []

for metric in metrics:
    # Filter the data
    df_filtered = df[df["Metric"] == metric]

    for model in models:
        df_filtered_model = df_filtered[df_filtered["Model"] == model]
        # Perform ANOVA
        f_stat, p_val = stats.f_oneway(df_filtered_model["low"],
                                       df_filtered_model["mid"],
                                       df_filtered_model["high"])

        # Append the results to the list
        results.append({
            "Model": model,
            "Metric": metric,
            "F-stat": f_stat,
            "p-val": p_val
        })

# Convert the list to a DataFrame
df_results = pd.DataFrame(results)

# Print the DataFrame
df_results

,Model,Metric,F-stat,p-val
0,DT,F1 Score,84.730576,3.344062e-25
1,RF,F1 Score,676.812892,6.947082e-75
2,LGBM,F1 Score,1284.974445,7.813330e-94
3,XGB,F1 Score,929.186853,3.858943e-84
4,DT,Precision,418.879634,1.941170e-61
5,RF,Precision,359.544707,2.437605e-57
6,LGBM,Precision,623.572732,1.553029e-72
7,XGB,Precision,541.967588,1.463990e-68
8,DT,Recall,5.674949,4.225875e-03
9,RF,Recall,306.719269,3.464268e-53
